In [129]:
from typing import Dict

def format_entities_for_prompt(entities: Dict[str, str]) -> str:
    """
    Converts a dictionary of entities into a formatted string representation.
    
    Args:
        entities: Dictionary containing entity key-value pairs
        
    Returns:
        Formatted string with entities listed in bullet points
    """
    formatted_str = ""
    for key, value in entities.items():
        # Convert snake_case to Title Case
        formatted_key = " ".join(word.capitalize() for word in key.split('_'))
        formatted_str += f"        - {formatted_key}: '{value}'\n"
    return formatted_str.rstrip()


def get_user_prompt(message: str, entities: Dict[str, str]) -> str:
    
    formatted_entities = format_entities_for_prompt(entities)
    
    user_prompt = f"""
    You are a friendly and helpful assistant. Please respond to the user's message in a casual and conversational way.

    # User's message
    {message}


    # General conversation:
    - Recognize whether the entities is ennough or not. Note that data range is required.
    - Maintain a friendly and helpful tone
    - Acknowledge user's request
    - Ask for clarification when needed
    - Use casual language while remaining professional
    - Make sure that you always must ask user to confirm the information before generating the report.

    # Current entities
    {formatted_entities}

    # Remember to check for required entities:
    - date_range (this information contains from_date and to_date)
    - product
    - product_detail
    - level
    - user
    
    # Make sure that you always must ask user to confirm the information before generating the report.

    # For example:
    ## User: Get me a Win Loss Detail Report yesterday
    ## Entities: 
        - Date Range: 'yesterday'
        - From Date: '02/04/2025'
        - To Date: '02/04/2025'
        - Product: 'All'
        - Product Detail: 'All'
        - Level: 'All'
        - User: 'All'
        
    ## Assistant:
        📊 Here is the Win Loss Detail Report for the date range at 02/04/2025:
            📖 Report Requested by: 'All'
            👤 Username: 'All'
            🏢 Product: 'All'
            📋 Product Detail: 'All'
            🎮 Level: 'All'
            📅 Date Range: 02/04/2025

        Alright, would you like to confirm this information to get report?
        
    ## User: Give me a Win Loss Detail report for Sportsbook from last week.
    ## Entities: 
        - Date Range: 'last week'
        - From Date: '27/03/2025'
        - To Date: '02/04/2025'
        - Product: 'All'
        - Product Detail: 'All'
        - Level: 'All'
        - User: 'All'
        
    ## Assistant:
        📊 Here is the Win Loss Detail Report for the date range from 27/03/2025 to 02/04/2025:
            📖 Report Requested by: 'All'
            👤 Username: 'All'
            🏢 Product: 'All'
            📋 Product Detail: 'All'
            🎮 Level: 'All'
            📅 Date Range: 27/03/2025 - 02/04/2025

        Alright, would you like to confirm this information to get report?
    
    ## User: Hello how are you today?
    ## Assistant: 
        👋 Hello! I'm a 🤖 friendly and helpful assistant. How can I assist you today? 😊
    
    
    ## User: Get me a Turnover Detail Report for Sportsbook Product
    ## Entities: 
        - Date Range: 'N/A'
        - From Date: 'N/A'
        - To Date: 'N/A'
        - Product: 'Sportsbook'
        - Product Detail: 'All'
        - Level: 'All'
        - User: 'All'
        
    ## Assistant:
        📊 Here is the Win Loss Detail Report for the date range from 02/04/2025 to 02/04/2025:
            📖 Report Requested by: 'All'
            👤 Username: 'All'
            🏢 Product: 'Sportsbook'
            📋 Product Detail: 'All'
            🎮 Level: 'All'
            📅 Date Range: 'N/A'
            
        ⚠️ However, I could not find any data for the date range, could you please provide the date range? 📅 Because this information is required for generating the report. 🔍
    
    Please respond to the user's message accordingly.
    """
    return user_prompt

In [130]:
import requests
import json

def llm_chat(
    system_prompt: str,
    user_prompt: str
) -> str:
    
    
    url = "https://ollama.selab.edu.vn/api/chat"
    
    headers = {
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "qwen2.5:14b",
        "messages": [
            {
                "role": "system", 
                "content": system_prompt
            },
            {
                "role": "user", 
                "content": user_prompt
            }
        ],
        'stream': False
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()



In [134]:
system_prompt = """
    You are a friendly and helpful assistant trained to handle conversations about Win/Loss reports and other queries in a casual yet professional manner.

    Your main responsibilities are:
    1. Help users get the information they need about Win/Loss reports
    2. Guide users to provide complete information when making requests
    3. Maintain a conversational and helpful tone
    4. Ask clarifying questions when information is missing
    5. Acknowledge user requests and confirm understanding

    Remember to:
    - Recognize whether the entities is enough or not. Note that data range is required.
    - Be friendly and approachable
    - Use natural, conversational language
    - Stay professional while being casual
    - Ask for missing information politely
    - Confirm understanding before proceeding
    - Make sure that you always must ask user to confirm the information before generating the report.
"""

user_prompt = get_user_prompt(
    "Get me a Win Loss on day 10", 
    {'date_range': 'day 10', 'from_date': '10/04/2025', 'to_date': '10/04/2025', 'product': 'All', 'product_detail': 'All', 'level': 'All', 'user': 'N/A'}
)

In [135]:
response = llm_chat(system_prompt, user_prompt)

In [136]:
print(response['message']['content'])

👋 Hey there! You want a Win Loss report for just day 10, right?

Here’s what I’ve got so far:
📅 Date Range: October 4, 2025 (both from and to date)

However, we need more details like the product, product detail, level, and user. Could you please provide these details? If not specified, I'll use 'All' for each.

Here’s a summary based on what we have:
📖 Report Requested by: N/A
👤 Username: N/A
🏢 Product: All
📋 Product Detail: All
🎮 Level: All
📅 Date Range: October 4, 2025

Would you like to confirm this information before I generate the report?
